# **與Google Drive連結**

In [ ]:
#連結Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#查看Google Drive的內容
!ls /content/gdrive/My\ Drive/TA/1091/crawler

file  Untitled0.ipynb


# **爬蟲的前置準備**

In [ ]:
#匯入常用的套件
import requests
import os
import re
import time
import urllib.request as ur
from bs4 import BeautifulSoup

In [ ]:
#定義全域變數
file_path = '/content/gdrive/My Drive/TA/1091/crawler/'
ptt_url_head = 'https://www.ptt.cc'

# **定義爬蟲所需要用到的函式**

In [ ]:
#取得網頁
def get_web_page(url):
  #取得網頁的回應
  response = requests.get(url=url, cookies={'over18':'1'})

  #如果回應的狀態碼是200以外的
  #代表網頁有問題
  if response.status_code != 200:
    print('Invalid url' + url)
    return None
  else:
    #如果回應的狀態碼是200代表網頁正常
    #回傳網頁的所有文字。
    return response.text

In [ ]:
#取得網頁的內容
def get_articles(web_page, date):
  #解析網頁
  soup = BeautifulSoup(web_page, 'html.parser')

  #取得上一頁按鈕的超連結
  a_tags = soup.find_all('a', 'btn wide')
  previous_page_url = a_tags[1]['href']

  #用來儲存取得的文章資料
  articles = []

  divs = soup.find_all('div', 'r-ent')
  for d in divs:
    #符合條件而且超連結存在，代表文章沒有被刪除。
    if d.find('div', 'date').text == date and d.find('a'):
      
      #取得文章的超連結
      href = d.find('a')['href']

      #取得文章的網頁
      content = get_web_page(ptt_url_head + href)
      content_soup = BeautifulSoup(content, 'html.parser')
      content_divs = content_soup.find_all('div', id='main-container')
      
      #用來儲存文章標題
      title = ''

      #用來儲存要下載的圖片超連結清單
      image_url_list = []

      #開始進行取得
      for cd in content_divs:
        #取得文章標題
        content_spans = cd.find_all('span', 'article-meta-value')
        #去除取得的文章標題的多餘空白
        title = content_spans[2].text.strip()

        #取得要下載的圖片超連結清單
        content_a_hrefs = cd.find_all('a',{'href':re.compile('https:\/\/(imgur|i\.imgur)\.com\/.*')})
        for cah in content_a_hrefs:
          #cah的長度不是0代表有取得圖片的超連結
          if len(cah) != 0:
            #這裡必須處理網址最後面沒有.jpg的情況
            if cah.text.find('.jpg') == -1:
              image_url_list.append(cah.text+'.jpg')
            else:
              image_url_list.append(cah.text)

      #儲存取得的文章標題與下載圖片的超連結清單
      articles.append({'title':title, 'image_url_list':image_url_list})

  #回傳文章資料與上一頁的超連結
  return articles, previous_page_url

In [ ]:
#儲存圖片
def save_images(folder_name, image_url_list):
  #將資料夾路徑與名稱串接起來
  folder_path = os.path.join(file_path,folder_name)
  
  #創建資料夾
  os.makedirs(folder_path)

  print('--------------------------------------------')
  print('Folder name:', folder_name)

  #使用例外處理不可預知的例外發生
  try:
    for image_url in image_url_list:
      #取得圖片超連結的最後一段做為儲存圖片的檔名
      #例如kes83e.jpg
      file_name = image_url.split('/')[-1]

      #透過超連結下載圖片
      ur.urlretrieve(image_url, os.path.join(folder_path, file_name))
      print('{} saved...'.format(file_name))
  except Exception as e:
    #發生例外時印出例外
    print(e)

# **開始執行爬蟲**

In [ ]:
#定義最一開始的網頁超連結
ptt_beauty_url = 'https://www.ptt.cc/bbs/Beauty/index.html'

#today是爬蟲的條件
#意思是只爬取指定日期的文章
today = time.strftime('%m', time.localtime()) + time.strftime('/%d', time.localtime())
print('--------------------------------------------')
print('Today is', today)

#開始進行爬蟲
print('--------------------------------------------')
print('Start')
while True:
  #呼叫get_web_page()函式取得網頁的所有文字
  page = get_web_page(ptt_beauty_url) 
  
  #如果page不是None就進行爬蟲
  if page:
    #呼叫get_articles()函式取得文章與上一頁的超連結
    articles, previous_page_link = get_articles(page, today)

    #如果articles不是None就下載圖片
    if articles:
      for article_content in articles:
        #文章標題作為資料夾名稱
        title = article_content['title']

        #取得要下載的圖片超連結清單
        image_url_list = article_content['image_url_list']

        #呼叫save_images()函式下載圖片
        save_images(title, image_url_list)
      
      #更新網址進行下一輪的爬蟲
      ptt_beauty_url = ptt_url_head + previous_page_link
    else:
      #如果articles是None代表沒有符合條件的文章了
      #此時可以停止爬蟲
      break
print('--------------------------------------------')
print('End')

--------------------------------------------
Today is 10/02
--------------------------------------------
Start
--------------------------------------------
Folder name: [正妹] 奶大還是臉正？抖幾？
WgFpFuu.jpg saved...
Ib7ImPt.jpg saved...
1yl49ib.jpg saved...
4z32Z7s.jpg saved...
--------------------------------------------
Folder name: [正妹] 攝影師真的很厲害
n0abslH.jpg saved...
45C9g6Y.jpg saved...
Sgupv6w.jpg saved...
HDyYZAB.jpg saved...
z8vktsk.jpg saved...
zdaGIyB.jpg saved...
ysIRopW.jpg saved...
6uu1wqS.jpg saved...
UzhA4BZ.jpg saved...
qtOmV1G.jpg saved...
SjSSKsl.jpg saved...
QJd1RKN.jpg saved...
--------------------------------------------
Folder name: [正妹] 大尺碼｜肉特(180)
jqUR4PG.jpg saved...
ezfFHEe.jpg saved...
byGgaSL.jpg saved...
OMqP2kf.jpg saved...
88d4Lqj.jpg saved...
cDqGgS1.jpg saved...
EhAopyJ.jpg saved...
NZTQKHd.jpg saved...
S2F1xyQ.jpg saved...
2DaukaB.jpg saved...
wKBIOhC.jpg saved...
S21OmV8.jpg saved...
pDxIhq4.jpg saved...
b7oYqKn.jpg saved...
giRdN66.jpg saved...
VZvBpuK.jpg saved